#  Токенизация и формирование датасетов в задачах NLP

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://huggingface.co/docs/tokenizers/index
* https://huggingface.co/docs/tokenizers/pipeline
* https://huggingface.co/docs/tokenizers/api/trainers#tokenizers.trainers.WordLevelTrainer
* Хороший минималистичный пакет с набором готовых преобразований, но больше не развивается:
    * https://pytorch.org/text/stable/
    * https://pytorch.org/text/stable/vocab.html
    * https://pytorch.org/text/stable/transforms.html

## Задачи для совместного разбора

1\. Рассмотрите основные шаги по преобразованию текста в последовательность индексов токенов.

In [1]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]

In [2]:
corpus = [
    doc.lower().split()
    for doc in corpus
]

In [3]:
corpus

[['студенты',
  'усердно',
  'занимаются',
  'стремясь',
  'получить',
  'знания',
  'и',
  'достичь',
  'успеха'],
 ['студенты',
  'активно',
  'участвуют',
  'в',
  'общественной',
  'жизни',
  'университета']]

In [4]:
words = set()
words.update(corpus[0])
words.update(corpus[1])
len(words)

15

In [5]:
stoi = {word: idx for idx, word in enumerate(words)}
stoi["<UNK>"] = len(stoi)
stoi["<PAD>"] = len(stoi)

In [6]:
corpus_i = [
    [stoi.get(word, stoi["<UNK>"]) for word in doc]
    for doc in corpus
]
corpus_i

[[1, 5, 7, 6, 4, 3, 13, 14, 8], [1, 11, 2, 9, 0, 10, 12]]

In [7]:
import torch

# torch.tensor(corpus_i)

2\. Обсудите основные требования к инструменту для построения набора индексов

3\. Рассмотрите пример работы с пакетом `tokenizers` для построения набора индексов токенов.

In [8]:
from tokenizers import Tokenizer

from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer

tokenizer = Tokenizer(WordLevel())
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(special_tokens=["<PAD>", "<UNK>"])

In [9]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]
tokenizer.train_from_iterator(corpus, trainer=trainer)

In [10]:
tokenizer.get_vocab()

{'достичь': 5,
 '<PAD>': 0,
 'усердно': 14,
 'получить': 11,
 'Студенты': 2,
 'стремясь': 12,
 'активно': 3,
 '<UNK>': 1,
 'знания': 8,
 'участвуют': 16,
 'занимаются': 7,
 'жизни': 6,
 'и': 9,
 'успеха': 15,
 'в': 4,
 'университета': 13,
 'общественной': 10}

In [11]:
encoders = tokenizer.encode(corpus[0])
encoders.ids

[2, 14, 7, 12, 11, 8, 9, 5, 15]

In [12]:
encoders.tokens

['Студенты',
 'усердно',
 'занимаются',
 'стремясь',
 'получить',
 'знания',
 'и',
 'достичь',
 'успеха']

In [13]:
tokenizer.enable_padding

<function Tokenizer.enable_padding(self, direction='right', pad_id=0, pad_type_id=0, pad_token='[PAD]', length=None, pad_to_multiple_of=None)>

## Задачи для самостоятельного решения

In [ ]:
from functools import reduce
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers.normalizers import Lowercase, Strip, Replace, Sequence as NormalizerSequence
from tokenizers import Regex
from tokenizers.processors import TemplateProcessing
from itertools import permutations


<p class="task" id="1"></p>

1\. Опишите класс `Vocab`. При создании объекта `Vocab` в конструктор передается набор текстов, предварительно разбитых на токены. Объект должен позволять:
* по токену получить его уникальный индекс (в случае отсутствия токена в словаре вернуть 1)
* по индексу токена получить сам токен (в случае отсутствия токена в словаре вернуть `<UNK>`)

Первые 4 индекса зарезервированы под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.
    
Создайте `Vocab` на основе списка `corpus` и закодируйте каждый токен в предложениях, используя `Vocab`. Выведите полученный результат на экран. Придумайте предложение со словом, которого не было в исходном корпусе текста, и продемонстрируйте на его примере процесс кодирования.

- [x] Проверено на семинаре

In [15]:
tokenizer = RegexpTokenizer(r"\w+")

In [16]:
class Vocab:
    def __init__(self, data: list[list[str]]) -> None:
        full = reduce(lambda x,y: x+y, data)        
        full = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']+list(set(full))
        self.stoi_ = {word:idx for idx, word in enumerate(full)}
        self.itos_ = {idx:word for idx, word in enumerate(full)}

    def itos(self, idx: int) -> str:
        """Возвращает токен по индексу"""
        return self.itos_.get(idx,self.itos_[1])
    
    def stoi(self, s: str) -> int:
        """Возвращает индекс токена"""
        return self.stoi_.get(s,self.stoi_['<UNK>'])

    def encode(self, text: str) -> list[int]:
        doc = tokenizer.tokenize(text.lower())
        return [self.stoi(word) for word in doc]

In [17]:
corpus = [
    "Маленький котенок игриво прыгает за шариком",
    "Пушистый котик мурлыкает, лежа на солнышке",
    "Котенок любопытно нюхает цветы в саду",
    "Котик ловко лазает по дереву, исследуя окружающий мир",
    "Спящий котик мило моргает своими яркими глазками",
]

In [18]:
corpus_preproc = [
    tokenizer.tokenize(doc.lower())
    for doc in corpus
]

voc = Vocab(corpus_preproc)

[voc.encode(s) for s in corpus]

[[6, 10, 25, 18, 28, 11],
 [31, 7, 30, 14, 8, 19],
 [10, 4, 26, 21, 15, 24],
 [7, 16, 5, 23, 9, 22, 27, 13],
 [20, 7, 12, 32, 17, 33, 29]]

In [19]:
voc.stoi_

{'<PAD>': 0,
 '<UNK>': 1,
 '<SOS>': 2,
 '<EOS>': 3,
 'любопытно': 4,
 'лазает': 5,
 'маленький': 6,
 'котик': 7,
 'на': 8,
 'дереву': 9,
 'котенок': 10,
 'шариком': 11,
 'мило': 12,
 'мир': 13,
 'лежа': 14,
 'в': 15,
 'ловко': 16,
 'своими': 17,
 'прыгает': 18,
 'солнышке': 19,
 'спящий': 20,
 'цветы': 21,
 'исследуя': 22,
 'по': 23,
 'саду': 24,
 'игриво': 25,
 'нюхает': 26,
 'окружающий': 27,
 'за': 28,
 'глазками': 29,
 'мурлыкает': 30,
 'пушистый': 31,
 'моргает': 32,
 'яркими': 33}

<p class="task" id="2"></p>

2\. Создайте класс `NewsDataset` на основе данных из файла `news.csv`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости $i$ и метку класса для этой новости. Для кодирования текстов используйте собственную реализацию `Vocab`. Набор индексов токенов возвращайте в виде тензора. Предусмотрите возможность передавать в конструктор класса преобразования.

Выведите на экран результат выполнения `dataset[0]`.

- [x] Проверено на семинаре

In [21]:
class NewsDataset:
    def __init__(self, filename, transforms=None):
        self.df = pd.read_csv(filename)
        self.news_vocab =  Vocab(self.df.text.apply(lambda x: tokenizer.tokenize(x.lower())).values.tolist())
        self.transforms = transforms
        
    def __getitem__(self, i):
        item = self.df.iloc[i]
        res = self.news_vocab.encode(item.text)
        if self.transforms:
            for i in self.transforms:
                res = i(res)
        return torch.tensor(res), item.label
    
    def __len__(self):
        return self.df.shape[0]
    
news = NewsDataset('data/news.csv')

news[1]

(tensor([5219, 3057, 5001, 4267,   76, 1964, 1545, 4603,  820]), np.int64(1))

<p class="task" id="3"></p>

3\. Реализуйте преобразование `PadTruncate`, которое:
- обрезает каждый текст в батче до `n` токенов, если текст был длиннее.
- расширяет каждый текст в батче до `n` токенов значением `pad_idx`, если текст был короче.


Создайте объект `NewsDataset` c применением данного преобразования при создании объекта. Создайте батч из нескольких примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент. Ваш пример должен показать все сценарии работы данного преобразования.

- [ ] Проверено на семинаре

pad_idx = 0

In [22]:
class PadTruncate:
    def __init__(self, n, pad_idx=0):
        self.n = n
        self.pad_idx = pad_idx
    
    def __call__(self, text):
        res = []
        if len(text)>=self.n:
            res.append(text[:self.n])
        else:
            res.append(text + [0]*(self.n - len(text)))
                
        return res


news = NewsDataset('data/news.csv', [PadTruncate(8)])
news_dataloader = torch.utils.data.DataLoader(news,4)
for batch in news_dataloader:
    print(batch)
    break


print("\n\nРазмеры компонент:")
print(batch[0].shape)
print(batch[1].shape)

[tensor([[[ 434,  140, 1123, 4149, 3043, 4340,    0,    0]],

        [[5219, 3057, 5001, 4267,   76, 1964, 1545, 4603]],

        [[ 434, 2343, 2444, 5772, 3483, 1964, 1545, 1932]],

        [[5827, 5843, 4425, 3569,  306, 5213,  581, 1099]]]), tensor([0, 1, 0, 1])]


Размеры компонент:
torch.Size([4, 1, 8])
torch.Size([4])


<p class="task" id="4"></p>

4\. Создайте объект `tokenizers.Tokenizer` на основе данных из файла `news.csv`. Для выделения токенов из текста используйте разбиение по пробелам (pre-tokenizer `Whitespace` + model `BPE` ). На этапе нормализации приводите текст к нижнему регистру и убирайте знаки препинания. Включите опцию паддинга. Первые 4 индекса зарезервируйте под специальные токены `[PAD]`, `[UNK]`, `[SOS]`, `[EOS]`.

Опишите класс `NewsDatasetHfTokenizer`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости $i$ и метку класса для этой новости. Для кодирования текстов используйте обученный токенизатор. Набор индексов токенов возвращайте в виде тензора. Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.

- [ ] Проверено на семинаре

In [23]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.normalizer = NormalizerSequence([
    Lowercase(),
    Replace(Regex(r"[^\w\s]"), "")
])
tokenizer.pre_tokenizer = Whitespace()

trainer = BpeTrainer(
    special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"],
    min_frequency=2,
    show_progress=True
)

tokenizer.train_from_iterator(news.df.text.values.tolist(), trainer=trainer)
tokenizer.enable_padding(
    pad_id=tokenizer.token_to_id("[PAD]"),
    pad_token="[PAD]",
    length=64
)
tokenizer.post_processor = TemplateProcessing(
    single="[SOS] $A [EOS]",
    pair="[SOS] $A [EOS] $B [EOS]",
    special_tokens=[
        ("[SOS]", tokenizer.token_to_id("[SOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")),
    ],
)

vocab = tokenizer.get_vocab()
vocab

{'ци': 113,
 'путин': 559,
 'русской': 3668,
 'миллиар': 693,
 'тки': 838,
 'курь': 1864,
 'рен': 501,
 'пней': 2233,
 'заблоки': 860,
 'электрон': 2828,
 'совершенно': 2815,
 'годом': 1777,
 'тельного': 2551,
 'миллиарда': 2692,
 'взлетно': 3745,
 'заморо': 1774,
 'адмирал': 4418,
 'да': 108,
 'ева': 1402,
 'r': 30,
 'пин': 1298,
 'могли': 3445,
 'отменить': 2663,
 'оде': 613,
 'причисли': 4261,
 'очень': 4453,
 'дах': 3377,
 'гани': 976,
 'концер': 1022,
 'тури': 2495,
 'убрать': 4329,
 'использует': 4279,
 'кризис': 3895,
 'действи': 2729,
 'честь': 1870,
 'бы': 208,
 'доб': 3004,
 'недружественных': 2850,
 'китайское': 4015,
 'офици': 1421,
 'до': 117,
 'замени': 1770,
 'известно': 1609,
 'попросил': 1158,
 'борца': 4003,
 'чё': 1724,
 'пи': 159,
 'конфли': 3611,
 'валификации': 3677,
 'син': 3279,
 'леса': 3331,
 'леонид': 4495,
 'сроки': 2675,
 'чения': 777,
 'великобритании': 1160,
 'жало': 1904,
 'расс': 310,
 'организации': 4124,
 'зен': 2200,
 'покушении': 3998,
 'коррупцией'

In [24]:
class NewsDatasetHfTokenizer(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = str(self.df.loc[idx, "text"])
        label = int(self.df.loc[idx, "label"])
        
        encoding = self.tokenizer.encode(text)
        input_ids = torch.tensor(encoding.ids, dtype=torch.long)
        
        return {
            "input_ids": input_ids,
            "label": torch.tensor(label, dtype=torch.long)
        }

In [25]:
dataset = NewsDatasetHfTokenizer(news.df, tokenizer)
dataloader = DataLoader(
    dataset,
    batch_size=16
    )

In [26]:
batch = next(iter(dataloader))
print(batch)
print("\n\nРазмеры компонент:")
print(f"input_ids: {batch['input_ids'].shape}")
print(f"label: {batch['label'].shape}")

{'input_ids': tensor([[   2,  350, 1374,  ...,    0,    0,    0],
        [   2,  304, 1153,  ...,    0,    0,    0],
        [   2,  350,  602,  ...,    0,    0,    0],
        ...,
        [   2,   75, 2037,  ...,    0,    0,    0],
        [   2,  492, 3807,  ...,    0,    0,    0],
        [   2, 1880, 1817,  ...,    0,    0,    0]]), 'label': tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1])}


Размеры компонент:
input_ids: torch.Size([16, 64])
label: torch.Size([16])


<p class="task" id="5"></p>

5\. Возьмите один тензор из батча, полученного в предыдущем задании. Используя методы обученного токенизатора, превратите индексы обратно в текст. Выведите на экран набор индексов токенов, набор токенов, исходный текст и декодированный текст.


- [ ] Проверено на семинаре

In [27]:
print('"base" - "tokenizer.decode"\n')

for i in range(2):
    print(f'"{dataset.df.text.loc[i]}" - "{tokenizer.decode(batch['input_ids'][i].tolist())}"')

"base" - "tokenizer.decode"

"лукашенко пригрозил литовским танкам белорусской картошкой " - "лукашенко пригрозил ли тов ским тан кам белорусской кар то ш кой"
"российские компании оказались в опасности из за глобального потепления" - "российские компании оказа лись в опасности из за глобаль ного поте п ления"


<p class="task" id="6"></p>

6\. Используя коэффициент Жаккара, найдите близость между каждой парой документов из предложенного списка. Напишите функцию `jaccard_similarity(text1, text2, tokenizer)`, которая токенизирует оба текста (превращает в множество уникальных токенов) и возвращает к-т Жаккара.

Сравните значение метрики при использовании Whitespace токенизатора (по словам) и вашего обученного BPE токенизатора.


- [ ] Проверено на семинаре

In [28]:
corpus = [
    "кот сидит на коврике",
    "собака сидит",
    "собака сидит на диване",
    "собака бежит"
]

In [29]:
def jaccard_similarity(text1, text2, tokenizer):    
    special_tokens = {"[PAD]", "[UNK]", "[SOS]", "[EOS]"}
    tokens1 = {t for t in set(tokenizer.encode(text1).tokens) if t.strip() and t not in special_tokens}
    tokens2 = {t for t in set(tokenizer.encode(text1).tokens) if t.strip() and t not in special_tokens}
    
    if not tokens1 and not tokens2:
        return 1.0 
    if not tokens1 or not tokens2:
        return 0.0
    
    intersection = tokens1 & tokens2
    union = tokens1 | tokens2
    return len(intersection) / len(union)

In [30]:
tokenizer_ws = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer_ws.pre_tokenizer = Whitespace()
trainer_ws = WordLevelTrainer(special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"])

tokenizer_ws.train_from_iterator(news.df.text.values.tolist(), trainer=trainer_ws)
tokenizer_ws.enable_padding(
    pad_id=tokenizer_ws.token_to_id("[PAD]"),
    pad_token="[PAD]",
    length=64
)

In [31]:
answer = []

for i in permutations(corpus, 2):
    answer.append(list(i)+[jaccard_similarity(*i, tokenizer), jaccard_similarity(*i, tokenizer_ws)])

pd.DataFrame(answer, columns=['text1','text1','jaccard_BPE', 'jaccard_WS'])

,text1,text1,jaccard_BPE,jaccard_WS
0,кот сидит на коврике,собака сидит,1.0,1.0
1,кот сидит на коврике,собака сидит на диване,1.0,1.0
2,кот сидит на коврике,собака бежит,1.0,1.0
3,собака сидит,кот сидит на коврике,1.0,1.0
4,собака сидит,собака сидит на диване,1.0,1.0
5,собака сидит,собака бежит,1.0,1.0
6,собака сидит на диване,кот сидит на коврике,1.0,1.0
7,собака сидит на диване,собака сидит,1.0,1.0
8,собака сидит на диване,собака бежит,1.0,1.0
9,собака бежит,кот сидит на коврике,1.0,1.0
